In [3]:
#=================================== importing the libraries ===================================

import pandas as pd
import numpy as np
import cPickle as pickle
#=================================== NLP Libraries ===================================
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tag import pos_tag
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import PunktSentenceTokenizer



In [5]:
# =================================================================================================
# Loading the clean'ed dataset
#
# ============================================================================================================

with open('indeed_job_detail_master_16k_df.p','rb') as f:
    master_df = pickle.load(f)

In [6]:
# =================================================================================================
#pulling out  titles and companies for Glass door
#
#============================================================================================================
listco = [str(x) for x in master_df.company_x.values.tolist()]
with open('listco.txt','wb') as f:
    f.write('\n'.join(listco))
    
titlez = [str(x) for x in master_df.title.values.tolist()]
with open('titlez.txt','wb') as f:
    f.write('\n'.join(titlez))    

In [7]:
# Check the size
print master_df.shape

(14669, 18)


In [8]:
# ============================================================================================================
# Clean Data sample
#
# ============================================================================================================

master_df.head(2)

,company_x,jobdesc,jobkey,location,snip,title,url,city,company_y,date,filename,formattedLocation,formattedLocationFull,jobtitle,latitude,longitude,source,state
0,Advocate Health Care,"As part of Advocate Health Care, Advocate Chil...",6244f7f3a48616a1,"Oak Lawn, IL 60453","\nAs part of Advocate Health Care, Advocate Ch...",Clinical Practice Specialist - 4 Hope,http://www.indeed.com/viewjob?jk=6244f7f3a4861...,Oak Lawn,Advocate Health Care,"Wed, 14 Sep 2016 13:15:06 GMT",CHI_jobdata.json,"Oak Lawn, IL","Oak Lawn, IL 60453",Clinical Practice Specialist - 4 Hope,41.719692,-87.74613,Advocate Health Care,IL
1,University of Washington Medical Center,The University of Washington (UW) is proud to ...,3d6a9f18f530167d,"Seattle, WA",\nThe University of Washington (UW) is proud t...,WEB DEVELOPER,http://www.indeed.com/viewjob?jk=3d6a9f18f5301...,Seattle,University of Washington Medical Center,"Tue, 30 Aug 2016 20:20:18 GMT",SEA_jobdata.json,"Seattle, WA","Seattle, WA",WEB DEVELOPER,47.604397,-122.32967,University of Washington Medical Center,WA


In [9]:
# ============================================================================================================
# EDA - look at the first 1000 rows of hte dataset
#
# ============================================================================================================

sample = master_df.head(1000)

In [10]:
# Set up stop works (words to ignore in the NLP analysis)
# runs a few minutes
from nltk.corpus import stopwords
stop = stopwords.words('english') + ['view','job','save','apply','email','friend', 'recommended', 'days','ago','30']

# running through the initial Count vectorizor of the worlds
cvec = CountVectorizer(stop_words=stop, decode_error='ignore', encoding='utf-8',lowercase=True,ngram_range=(2,4))
cvec.fit(sample['snip'])

#storing in a dataframe
cdf  = pd.DataFrame(cvec.transform(sample['snip']).todense(),
             columns=cvec.get_feature_names())



In [11]:
cdf.sum().sort_values(ascending=False).head(30)

business analyst              788
years experience              518
data analyst                  414
communication skills          411
bachelor degree               410
new york                      376
analyst jobs                  333
atlanta ga                    318
analyst salaries              313
york ny                       307
new york ny                   307
problem solving               304
computer science              290
equal opportunity             288
experience working            284
big data                      284
ability work                  270
san francisco                 269
project management            254
skills ability                253
business intelligence         245
opportunity employer          233
machine learning              228
chicago il                    225
indeed easily                 224
seattle wa                    223
san francisco ca              220
francisco ca                  220
data analytics                218
equal opportun

In [13]:
# compare using TFIDF
tvec = TfidfVectorizer(stop_words=stop, decode_error='ignore', encoding='utf-8',lowercase=True, ngram_range=(2,5))
tvec.fit(sample['snip'])

tdf  = pd.DataFrame(tvec.transform(sample['snip']).todense(),
             columns=tvec.get_feature_names())

In [14]:
tdf.sum().sort_values(ascending=False).head(30)

business analyst         7.664473
new york                 4.839338
data analyst             4.716925
atlanta ga               4.434705
york ny                  4.253951
new york ny              4.253951
years experience         3.832588
san francisco            3.630345
seattle wa               3.274637
san francisco ca         3.245878
francisco ca             3.245878
big data                 3.180232
chicago il               3.033002
communication skills     3.000277
business intelligence    2.974508
machine learning         2.881722
bachelor degree          2.881003
software engineer        2.730499
experience working       2.709834
minneapolis mn           2.706005
analyst jobs             2.637627
computer science         2.612515
analyst salaries         2.557712
data engineer            2.527648
problem solving          2.468091
ability work             2.459743
los angeles              2.451944
washington dc            2.348517
please review            2.327676
project manage

In [15]:
# testing the parts of speech analysis
tok = WordPunctTokenizer()
pos_tag(tok.tokenize("today is a great day to learn nlp"))

[('today', 'NN'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('great', 'JJ'),
 ('day', 'NN'),
 ('to', 'TO'),
 ('learn', 'VB'),
 ('nlp', 'NN')]

In [21]:
# look at a single sample and see if we can
# 1.) break by sentence
# 2.) break by parts of speech into phrases
# 3.) then into words
print sample['snip'][8]



Solution ArchitectFairfax, VAAbout ICF's Information Insights (I2) GroupICF's Information Insights (I2) team is comprised of experienced professionals with the skills required to unlock the power of governed data discovery. Our staff provides deep technical and business support for data acquisition, data analysis/data science, and interactive data visualizations.Job Description: We're looking for a Solution Architect who wants to be responsible for defining technology solutions. You'll integrate different domain applications and data in ways that enhance our clients' current environments and helps pushes them forward to next generation capabilities. This will include helping them build and support data science environments in AWS as well as applying big data analytics in a coordinated, cohesive manner to help clients solve business use cases they could not previously address.What you'll be doing: * Develop long-term partnerships with customers and vendors and contribute to the develop

In [23]:
#try to cleanup the text -- any type of spaces / etc.
print 
decoded_str = sample['jobdesc'][8].decode("windows-1252")
encoded_str = decoded_str.encode("utf8").replace("â€™","'")

print encoded_str



Solution ArchitectFairfax, VAAbout ICF's Information Insights (I2) GroupICF's Information Insights (I2) team is comprised of experienced professionals with the skills required to unlock the power of governed data discovery. Our staff provides deep technical and business support for data acquisition, data analysis/data science, and interactive data visualizations.Job Description: We're looking for a Solution Architect who wants to be responsible for defining technology solutions. You'll integrate different domain applications and data in ways that enhance our clients' current environments and helps pushes them forward to next generation capabilities. This will include helping them build and support data science environments in AWS as well as applying big data analytics in a coordinated, cohesive manner to help clients solve business use cases they could not previously address.What you'll be doing: * Develop long-term partnerships with customers and vendors and contribute to the develop

In [25]:
#view the paragraph as sentences
sent_detector = PunktSentenceTokenizer()
sent_detector.sentences_from_text(encoded_str)


["Solution ArchitectFairfax, VAAbout ICF's Information Insights (I2) GroupICF's Information Insights (I2) team is comprised of experienced professionals with the skills required to unlock the power of governed data discovery.",
 "Our staff provides deep technical and business support for data acquisition, data analysis/data science, and interactive data visualizations.Job Description: We're looking for a Solution Architect who wants to be responsible for defining technology solutions.",
 "You'll integrate different domain applications and data in ways that enhance our clients' current environments and helps pushes them forward to next generation capabilities.",
 "This will include helping them build and support data science environments in AWS as well as applying big data analytics in a coordinated, cohesive manner to help clients solve business use cases they could not previously address.What you'll be doing: * Develop long-term partnerships with customers and vendors and contribute t

In [27]:
# look at one of the sentences 
tok_sample = pos_tag(tok.tokenize(sample['jobdesc'][8]))
tok_sample

[('Solution', 'NNP'),
 ('ArchitectFairfax', 'NNP'),
 (',', ','),
 ('VAAbout', 'NNP'),
 ('ICF', 'NNP'),
 ("'", 'POS'),
 ('s', 'JJ'),
 ('Information', 'NNP'),
 ('Insights', 'NNP'),
 ('(', '('),
 ('I2', 'NNP'),
 (')', ')'),
 ('GroupICF', 'NNP'),
 ("'", 'POS'),
 ('s', 'JJ'),
 ('Information', 'NNP'),
 ('Insights', 'NNP'),
 ('(', '('),
 ('I2', 'NNP'),
 (')', ')'),
 ('team', 'NN'),
 ('is', 'VBZ'),
 ('comprised', 'VBN'),
 ('of', 'IN'),
 ('experienced', 'JJ'),
 ('professionals', 'NNS'),
 ('with', 'IN'),
 ('the', 'DT'),
 ('skills', 'NNS'),
 ('required', 'VBN'),
 ('to', 'TO'),
 ('unlock', 'VB'),
 ('the', 'DT'),
 ('power', 'NN'),
 ('of', 'IN'),
 ('governed', 'VBN'),
 ('data', 'NNS'),
 ('discovery', 'NN'),
 ('.', '.'),
 ('Our', 'PRP$'),
 ('staff', 'NN'),
 ('provides', 'VBZ'),
 ('deep', 'JJ'),
 ('technical', 'JJ'),
 ('and', 'CC'),
 ('business', 'NN'),
 ('support', 'NN'),
 ('for', 'IN'),
 ('data', 'NNS'),
 ('acquisition', 'NN'),
 (',', ','),
 ('data', 'NNS'),
 ('analysis', 'NN'),
 ('/', 'NNP'),
 ('da

In [29]:
# All nouns
print [x for x in tok_sample if x[1]=='NNP']

[('Solution', 'NNP'), ('ArchitectFairfax', 'NNP'), ('VAAbout', 'NNP'), ('ICF', 'NNP'), ('Information', 'NNP'), ('Insights', 'NNP'), ('I2', 'NNP'), ('GroupICF', 'NNP'), ('Information', 'NNP'), ('Insights', 'NNP'), ('I2', 'NNP'), ('/', 'NNP'), ('Job', 'NNP'), ('Description', 'NNP'), ('Solution', 'NNP'), ('Architect', 'NNP'), ('AWS', 'NNP'), ('Develop', 'NNP'), ('.*', 'NNP'), ('Develop', 'NNP'), ('Work', 'NNP'), ('Bachelor', 'NNP'), ('Computer', 'NNP'), ('Science', 'NNP'), ('Engineering', 'NNP'), ('.*', 'NNP'), ('Significant', 'NNP'), ('/', 'NNP'), ('Big', 'NNP'), ('Data', 'NNP'), ('Hadoop', 'NNP'), ('MongoDB', 'NNP'), ('),', 'NNP'), ('/', 'NNP'), ('Significant', 'NNP'), ('ETL', 'NNP'), ('MDM', 'NNP'), ('EAI', 'NNP'), ('Web', 'NNP'), ('Service', 'NNP'), (').*', 'NNP'), ('Ability', 'NNP'), ('Skills', 'NNP'), ('Agile', 'NNP'), ('Software', 'NNP'), ('Delivery', 'NNP'), ('Life', 'NNP'), ('Cycle', 'NNP'), ('.*', 'NNP'), ('Experience', 'NNP'), ('Vendor', 'NNP'), ('Management', 'NNP'), ('Process

In [30]:
# All verbs
print [x for x in tok_sample if 'V' in x[1]]

[('is', 'VBZ'), ('comprised', 'VBN'), ('required', 'VBN'), ('unlock', 'VB'), ('governed', 'VBN'), ('provides', 'VBZ'), ('re', 'VBP'), ('looking', 'VBG'), ('wants', 'VBZ'), ('be', 'VB'), ('defining', 'VBG'), ('enhance', 'VBP'), ('helps', 'VBZ'), ('pushes', 'VB'), ('include', 'VB'), ('helping', 'VBG'), ('build', 'VB'), ('support', 'VB'), ('applying', 'VBG'), ('help', 'VB'), ('solve', 'VB'), ('be', 'VB'), ('doing', 'VBG'), ('understanding', 'VBG'), ('working', 'VBG'), ('facilitate', 'VB'), ('including', 'VBG'), ('preparing', 'VBG'), ('be', 'VB'), ('creating', 'VBG'), ('delivered', 'VBN'), ('.*', 'VBP'), ('translate', 'VB'), ('including', 'VBG'), ('assisting', 'VBG'), ('planning', 'VBG'), ('looping', 'VBG'), ('address', 'VB'), ('confirming', 'VBG'), ('are', 'VBP'), ('followed', 'VBN'), ('related', 'VBN'), ('.*', 'VBZ'), ('developing', 'VBG'), ('defining', 'VBG'), ('implementing', 'VBG'), ('including', 'VBG'), ('reporting', 'VBG'), ('obtain', 'VB'), ('clearancePreferred', 'VBN'), ('driving'